# Imports

In [1]:
from PAL import *
engine = Redshift_Connection('ssopariwalla')

# schema = account_schema_mapping['AMZ']
schema = 'halo_connect_customer_data'


# Load AMZ Facility List

In [2]:
facility_list = pd.read_csv('Amazon Facility List.csv',usecols=['name','latitude','longitude'])


# Build Query to Flag Each Vehicles Closest Distance to an Amazon Site in it's last 3 days of driving.

In [7]:
# build one SELECT per row
selects = []
for name, lat, lon in facility_list.itertuples(index=False):
    safe_name = str(name).replace("'", "''")
    selects.append(
        f"SELECT '{safe_name}' AS name, {lat:.6f} AS latitude, {lon:.6f} AS longitude, 1"
    )

# join them with UNION ALL
union_block = "\nUNION ALL\n".join(selects)

# plug into your CTE
query = f"""
WITH amazon_facilities(name, latitude, longitude) AS (
    {union_block}
),
latest_pressure_reading_timestamp AS (
    SELECT
        vehicle_meta_data.fleet_vehicle_id,
        MAX(sensor_data.pressure_date) AS last_reading
      FROM 
          {schema}.vehicle_meta_data INNER JOIN {schema}.meta_data
              ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
          INNER JOIN {schema}.sensor_data
              ON sensor_data.meta_data_id = meta_data.id
      GROUP BY 
          vehicle_meta_data.fleet_vehicle_id
),
vehicle_loc_cross_amz_loc_w_dist AS (
SELECT DISTINCT
    fleet_meta_data.code,
    fleet_meta_data.fleet_name,
    vehicle_meta_data.fleet_vehicle_id,
    vehicle_meta_data.vehicle_id,
    LPRT.last_reading,
    CASE 
        WHEN fleet_meta_data.fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END AS maint_provider,
    sensor_data.pressure_date,
    sensor_data.latitude as vehicle_latitude,
    sensor_data.longitude as vehicle_longitude,
      3959 * ACOS(
    COS(RADIANS(amazon_facilities.latitude))
    * COS(RADIANS(sensor_data.latitude))
    * COS(
      RADIANS(sensor_data.longitude)
      - RADIANS(amazon_facilities.longitude)
    )
    + SIN(RADIANS(amazon_facilities.latitude))
    * SIN(RADIANS(sensor_data.latitude))
  ) AS distance_miles_to_amz_facility
FROM
    {schema}.fleet_meta_data INNER JOIN {schema}.vehicle_meta_data
        ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
    INNER JOIN {schema}.meta_data 
        ON meta_data.vehicle_id = vehicle_meta_data.vehicle_id
    INNER JOIN {schema}.sensor_data
        ON sensor_data.meta_data_id = meta_data.id
    LEFT JOIN latest_pressure_reading_timestamp LPRT
        ON LPRT.fleet_vehicle_id = vehicle_meta_data.fleet_vehicle_id
    INNER JOIN amazon_facilities 
        ON (amazon_facilities.latitude  BETWEEN sensor_data.latitude  - 1.0  AND sensor_data.latitude  + 1.0
        AND amazon_facilities.longitude BETWEEN sensor_data.longitude - 1.0  AND sensor_data.longitude + 1.0)
WHERE
    fleet_meta_data.code NOT IN ('Y87BQV9', 'X77OKO7', '1L10O39N', '1GOPAPMF', '3FF99DS', '1FBR7DGI', '1M5190AC', '19RVO6ZP')
    AND fleet_meta_data.status = 'ACTIVE'
    AND sensor_data.pressure_date >= DATEADD(day, -3, LPRT.last_reading) )

SELECT
    code,
    fleet_name,
    fleet_vehicle_id,
    vehicle_id,
    maint_provider,
    MIN(distance_miles_to_amz_facility)
FROM
    vehicle_loc_cross_amz_loc_w_dist
GROUP BY
    code,
    fleet_name,
    fleet_vehicle_id,
    vehicle_id,
    maint_provider
"""

with open("geofencing_query.txt", "w") as file:
    file.write(query)

In [ ]:
# now run it
# df = pd.read_sql(query, engine)
# df.to_csv('df.csv',index = False)